# Import libraries

In [1]:
import cv2
import operator
import numpy as np
from mtcnn import MTCNN
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load models

In [2]:
age_sex_models = keras.models.load_model('./age_sex_model.hdf5')

facial_model = keras.models.load_model('./facial_exprassion_model.hdf5')

# Preprocessing inputs functions

In [3]:
def gender(s):
    if float(s) < 0.4:
        return str('man')
    else:
        return str('woman')

def facial_pred_decode(list_):
#     dictionary = {0:'anger', 1:'contempt', 2:'disgust', 3:'fear', 4:'happy', 5:'sadness',6:'surprise'}
    dictionary = {0:'anger', 1:'disgust', 2:'fear', 3:'happy', 4:'natural', 5:'sad',6:'surprise'}
    _ , min_value = min(enumerate(list_), key=operator.itemgetter(1))
    max_index_1, _ = max(enumerate(list_), key=operator.itemgetter(1))
    first_rank = dictionary[max_index_1]
    list_[max_index_1] = min_value
    max_index_2, _ = max(enumerate(list_), key=operator.itemgetter(1))
    second_rank = dictionary[max_index_2]
    return  first_rank , second_rank

def face_detect(img_path):
    img = cv2.imread(img_path)
    model = MTCNN()
    face = model.detect_faces(img)
    boundingbox = face[0]['box']
    img = img[boundingbox[1]+10:boundingbox[1]+boundingbox[2]+20 , boundingbox[0]-10:boundingbox[0]+boundingbox[3]-10]
    return img

def resize(img , height , width):
    img = tf.image.resize(img , (height , width) , method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return img

def normalize(image):
    image = (image / 255)
    return image

def load_img(image_path):
    main = cv2.imread(image_path)
    img = face_detect(image_path)
#     img = tf.image.decode_image(img , 3 ,expand_animations=False)
    img = tf.cast(img , tf.float32)
    img = resize(img , 224 , 224)
    img = normalize(img)
    return main , img

def load_img_realtime(img):
    img = tf.cast(img , tf.float32)
    img = resize(img , 224 , 224)
    img = normalize(img)
    return img

# Infrence

In [4]:
def infrence(img_path):
    main ,img = load_img(img_path)
    img_d = tf.expand_dims(img, axis=0)
#     print('main shape is ',main.shape)
#     print('dim shape is ',img_d.shape)
    pred_sex_age = age_sex_models.predict(img_d)
    pred_facial = facial_model.predict(img_d)
### get predicate labels
    age_label = float(pred_sex_age[0][0][0])
    sex_label = gender(int(pred_sex_age[1][0][0]))
    facial = facial_pred_decode(pred_facial[0])
#     plt.imshow(face_detect(img_path))
#     plt.show()
    plt.imshow(main , cmap='gray')
    plt.show()
#     print(' sex = {}\n age = {}\n facial = {}'.format(sex_label , np.round(age_label) , facial))
    return sex_label , np.round(age_label) , facial

In [5]:
infrence('./image/test4.png')

UnimplementedError: Graph execution error:

Detected at node 'model/conv2d/Conv2D' defined at (most recent call last):
    File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/oem/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 612, in start
      self.io_loop.start()
    File "/usr/lib/python3/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.9/asyncio/base_events.py", line 596, in run_forever
      self._run_once()
    File "/usr/lib/python3.9/asyncio/base_events.py", line 1890, in _ru

# Real Time part

In [ ]:
# def model(img):
#     img = load_img_realtime(img)
#     img_d = tf.expand_dims(img, axis=0)
#     pred_sex_age = age_sex_models.predict(img_d)
#     pred_facial = facial_model.predict(img_d)
# ### get predicate labels
#     age_label = float(pred_sex_age[0][0][0])
#     sex_label = gender(int(pred_sex_age[1][0][0]))
#     facial = facial_pred_decode(pred_facial[0])
#     return sex_label , np.round(age_label) , facial


# cap = cv2.VideoCapture(0)
# face = cv2.CascadeClassifier('harrcascade_frontalface_default.xml')
# while 1:
#     ret,frame = cap.read()
#     if ret:
#         face1 = face.detectMultiScale(frame)
#         for x,y,w,h in face1:
#             img = frame[y:y+w , x:x+h]
#             sex , age , facial = model(img)
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)      
#             cv2.putText(img=frame, text=sex, org=(x+20, y),
#                     fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 255, 0), thickness=2)
#             cv2.putText(img=frame, text=str(age), org=(x+40, y),
#                                         fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 0, 255),thickness=2)
#             cv2.putText(img=frame, text=facial , org=(x+60, y),
#                     fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 0),thickness=2)
#             cv2.imshow('frame', frame)

#             k = cv2.waitKey(33)
#             if k == -1:
#                 continue
#             else:
#                 break
# cap.release()
# cv2.destroyAllWindows()   